In [367]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import datetime as dt

Import Data

In [368]:
#import data

df = pd.read_csv("data\produkt_tu_stunde_19480101_20211231_05906.txt", delimiter=';')
df

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor
0,5906,1948010101,5,-0.3,90.0,eor
1,5906,1948010102,5,-0.3,89.0,eor
2,5906,1948010103,5,0.1,89.0,eor
3,5906,1948010104,5,0.5,91.0,eor
4,5906,1948010105,5,0.8,87.0,eor
...,...,...,...,...,...,...
648607,5906,2021123119,3,10.2,91.0,eor
648608,5906,2021123120,3,10.9,88.0,eor
648609,5906,2021123121,3,11.0,88.0,eor
648610,5906,2021123122,3,11.1,88.0,eor


Clean & harmonize data set

In [369]:
#remove unneeded data

df=df.drop(columns=['STATIONS_ID', 'QN_9', 'eor'])
df.head(2)

,MESS_DATUM,TT_TU,RF_TU
0,1948010101,-0.3,90.0
1,1948010102,-0.3,89.0


In [370]:
#rename MESS_DATUM in Date

df = df.rename(columns={"MESS_DATUM": "Date"})

In [371]:
#convert int to date

df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d%H')
df.head(2)

,Date,TT_TU,RF_TU
0,1948-01-01 01:00:00,-0.3,90.0
1,1948-01-01 02:00:00,-0.3,89.0


In [372]:
#only last 21 years

df=df[df['Date'].dt.year >= 2001]
df

,Date,TT_TU,RF_TU
464615,2001-01-01 00:00:00,-2.1,87.0
464616,2001-01-01 01:00:00,-2.1,87.0
464617,2001-01-01 02:00:00,-1.2,82.0
464618,2001-01-01 03:00:00,0.0,75.0
464619,2001-01-01 04:00:00,0.5,71.0
...,...,...,...
648607,2021-12-31 19:00:00,10.2,91.0
648608,2021-12-31 20:00:00,10.9,88.0
648609,2021-12-31 21:00:00,11.0,88.0
648610,2021-12-31 22:00:00,11.1,88.0


In [373]:
#remove leap years from dataset

df = df[~((df.Date.dt.month == 2) & (df.Date.dt.day == 29)) ]

In [374]:
#problem with -999 values

df.min()

Date     2001-01-01 00:00:00
TT_TU                   -999
RF_TU                   -999
dtype: object

In [375]:
#replace -999 values with nan

df = df.replace(-999, np.nan)
df.min()

Date     2001-01-01 00:00:00
TT_TU                  -17.9
RF_TU                     11
dtype: object

In [376]:
#count the nan values

df.apply(pd.isnull).sum()

Date       0
TT_TU    151
RF_TU    164
dtype: int64

In [377]:
#fill the nan values with the values forwoard

df=df.ffill(axis = 0)
df.apply(pd.isnull).sum()


Date     0
TT_TU    0
RF_TU    0
dtype: int64

In [378]:
#d

df['Date'] = pd.to_datetime(df['Date']).dt.date
df2 = pd.DataFrame(columns= ["Date"]) 
df2["min_TT_TU"] = df.groupby(["Date"])["TT_TU"].min()
df2["max_TT_TU"] = df.groupby(["Date"])["TT_TU"].max()
df2["min_RF_TU"] = df.groupby(["Date"])["RF_TU"].min()
df2["max_RF_TU"] = df.groupby(["Date"])["RF_TU"].max()


df2['Date'] = df2.index
df2 = df2.reset_index(drop=True)
df2


,Date,min_TT_TU,max_TT_TU,min_RF_TU,max_RF_TU
0,2001-01-01,-2.1,3.1,69.0,87.0
1,2001-01-02,1.9,8.5,81.0,90.0
2,2001-01-03,1.9,8.3,73.0,90.0
3,2001-01-04,2.4,8.3,74.0,90.0
4,2001-01-05,6.6,11.1,75.0,89.0
...,...,...,...,...,...
7658,2021-12-27,3.2,10.4,79.0,100.0
7659,2021-12-28,7.7,11.0,74.0,98.0
7660,2021-12-29,8.7,10.7,76.0,99.0
7661,2021-12-30,11.0,15.1,81.0,98.0


Data set label & split

In [ ]:
#create labeled data set for predictions in one year  -> 365*24

prediction_hours = (1)
df["Prediction_TT"]=df["TT_TU"].shift(-prediction_hours)
df["Prediction_RF"]=df["RF_TU"].shift(-prediction_hours)

df

In [ ]:
#split the data in x und y axis

X = np.array(df[["TT_TU"]])
X = X[: len(df)-prediction_hours]

In [ ]:
#create y values

y = np.array(df["Prediction_TT"])
y = y[:-prediction_hours]

In [ ]:
#aufteilen der Daten in Trainings und Test Daten

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [ ]:
#erstellen einer varaible mit letzten 365*24 Werten aus dem df

prediction_hours_array = np.array(df[['TT_TU']])[-prediction_hours:]

Support Vector Regression